In [42]:
import math
import re
import pandas as pd
import numpy as np
import pymorphy2
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import preprocessing


def merge_com(data):
    data = data.copy()
    for i in range(data.shape[0]):
        data.loc[i, 'com_len'] = len(data.loc[i, 'comment'])
        
        try:
            data.loc[i, 'comPos_len'] = len(data.loc[i, 'commentPositive'])
        except TypeError:
            data.loc[i, 'comPos_len'] = 0
            
        try:
            data.loc[i, 'comNeg_len'] = len(data.loc[i, 'commentNegative'])
        except TypeError:
            data.loc[i, 'comNeg_len'] = 0
            
        data.loc[i, 'nCapital'] = len(re.findall("[A-Z]", data.loc[i, 'comment']))
        
        try:
            data.loc[i, 'nCapitalPos'] = len(re.findall("[A-Z]", data.loc[i, 'commentPositive']))
        except TypeError:
            data.loc[i, 'nCapitalPos'] = 0
            
        try:
            data.loc[i, 'nCapitalNeg'] = len(re.findall("[A-Z]", data.loc[i, 'commentNegative']))
        except TypeError:
            data.loc[i, 'nCapitalNeg'] = 0
        
        try:
            if math.isnan(data.commentNegative[i]):
                data.loc[i, 'hasNegComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.loc[i, 'comment'] + ' ' + data.loc[i, 'commentNegative']
            data.loc[i, 'hasNegComment'] = 1
        
        try:
            if math.isnan(data.commentPositive[i]):
                data.loc[i, 'hasPosComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.comment[i] + ' ' + data.commentPositive[i]
            data.loc[i, 'hasPosComment'] = 1
        
        data.loc[i, 'n_of_exc'] = data.loc[i, 'comment'].count('!')

    return data.drop(['commentNegative', 'commentPositive'], axis=1)

morph = MorphAnalyzer()
mystopwords = list(set(stopwords.words('russian')) - set(['много','без','никогда' , 'совсем' , 'не', 'нет', 'более','больше',  'ничего', "но", "хорошо", "лучше"])) + ['это',"х","р", "тыс", "тыщ", "руб"] 

def parse_sentence(sent):
    sent = sent.replace('...', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
    merged_text = ''.join(ch for ch in sent if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
#    tokens = [t_ for t_ in tokens if t_ not in mystopwords]
#    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(ch for ch in tokens)


print('Preparing dataset\n')
df = pd.read_csv('X_train.csv').drop(['userName', 'property', 'date'], axis=1)
df = merge_com(df)
df['parsed'] = df.comment.apply(parse_sentence)
df = df.drop(['comment'], axis=1)
df.reting = df.reting.apply(round)

#print('Splitting dataset into train and test\n')
#X_train, X_test, y_train, y_test = train_test_split(df.drop(['reting'], axis=1), 
#                                                    df['reting'], test_size=0.33)

tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(df['parsed'])

tfidf_train = tfidf.transform(df['parsed'])
#tfidf_test = tfidf.transform(X_test['parsed'])

print('Training model...\n')
clf2 = SVC(C = 1.0, kernel = 'linear').fit(tfidf_train, df['reting'])
#print('Score:')
#print(clf.score(tfidf_test, np.array(y_test)))

Preparing dataset

Training model...



In [45]:
print('Preparing dataset\n')
df = pd.read_csv('X_final_test.csv').drop(['userName', 'property', 'date'], axis=1)
df = merge_com(df)
df['parsed'] = df.comment.apply(parse_sentence)
df = df.drop(['comment'], axis=1)
#df.reting = df.reting.apply(round)

Preparing dataset



In [47]:
tfidf_test = tfidf.transform(df['parsed'])

In [49]:
df['rating'] = clf2.predict(tfidf_test)

In [51]:
df2 = pd.read_csv('X_final_test.csv')

In [52]:
df2['rating'] = clf2.predict(tfidf_test)

In [53]:
df2.to_csv('X_test_predicted.csv', index = Fal)

,comment,rating
0,"Согласна с Сашей и Наташей. Самая лучшая,самая...",5
1,"Хороший пылесос. Вроде качественный. Да, шумно...",5
2,"Пользовалась им почти 2 года, без нареканий. Н...",4
3,До этого у меня был триммер всего с одной наса...,5
4,Удобны в использовании. Волосы становятся глад...,5
5,ЖКашка достойная внимания.Картинка при сравнен...,4
6,"Красивая, бежевая, немаркая, ничего лишнего!",5
7,"не что не разглаживает, пар то что он выдает о...",1
8,"Отличный тостер, верой и правдой служит мне уж...",5
9,"Машинка шила исправно, но после того как я про...",3


In [41]:
print(clf.score(tfidf_train, df['reting']))

0.867068711105


In [43]:
print(clf2.score(tfidf_train, df['reting']))

0.893372682364


In [25]:
import math
import re
import pandas as pd
import numpy as np
import pymorphy2
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import preprocessing


def merge_com(data):
    data = data.copy()
    for i in range(data.shape[0]):
        data.loc[i, 'com_len'] = len(data.loc[i, 'comment'])
        
        try:
            data.loc[i, 'comPos_len'] = len(data.loc[i, 'commentPositive'])
        except TypeError:
            data.loc[i, 'comPos_len'] = 0
            
        try:
            data.loc[i, 'comNeg_len'] = len(data.loc[i, 'commentNegative'])
        except TypeError:
            data.loc[i, 'comNeg_len'] = 0
            
        data.loc[i, 'nCapital'] = len(re.findall("[A-Z]", data.loc[i, 'comment']))
        
        try:
            data.loc[i, 'nCapitalPos'] = len(re.findall("[A-Z]", data.loc[i, 'commentPositive']))
        except TypeError:
            data.loc[i, 'nCapitalPos'] = 0
            
        try:
            data.loc[i, 'nCapitalNeg'] = len(re.findall("[A-Z]", data.loc[i, 'commentNegative']))
        except TypeError:
            data.loc[i, 'nCapitalNeg'] = 0
        
        try:
            if math.isnan(data.commentNegative[i]):
                data.loc[i, 'hasNegComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.loc[i, 'comment'] + ' ' + data.loc[i, 'commentNegative']
            data.loc[i, 'hasNegComment'] = 1
        
        try:
            if math.isnan(data.commentPositive[i]):
                data.loc[i, 'hasPosComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.comment[i] + ' ' + data.commentPositive[i]
            data.loc[i, 'hasPosComment'] = 1
        
        data.loc[i, 'n_of_exc'] = data.loc[i, 'comment'].count('!')

    return data.drop(['commentNegative', 'commentPositive'], axis=1)

morph = MorphAnalyzer()
mystopwords = list(set(stopwords.words('russian')) - set(['много','без','никогда' , 'совсем' , 'не', 'нет', 'более','больше',  'ничего', "но", "хорошо", "лучше"])) + ['это',"х","р", "тыс", "тыщ", "руб"] 

def parse_sentence(sent):
    sent = sent.replace('...', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
    merged_text = ''.join(ch for ch in sent if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
    #tokens = [t_ for t_ in tokens if t_ not in mystopwords]
    #tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(ch for ch in tokens)


print('Preparing dataset\n')
df = pd.read_csv('X_train.csv').drop(['userName', 'property', 'date'], axis=1)
df = merge_com(df)
df['parsed'] = df.comment.apply(parse_sentence)
df = df.drop(['comment'], axis=1)
df.reting = df.reting.apply(round)

Preparing dataset



In [14]:
df.to_csv("data_parsed_not_norm.csv", index=False)

In [36]:
print('Splitting dataset into train and test\n')
X_train, X_test, y_train, y_test = train_test_split(df.drop(['reting'], axis=1), 
                                                    df['reting'], test_size=0.33)

tfidf = TfidfVectorizer(analyzer = 'word', ngram_range=(1,3)).fit(X_train['parsed'])

Splitting dataset into train and test



In [37]:
tfidf_train = tfidf.transform(X_train['parsed'])
tfidf_test = tfidf.transform(X_test['parsed'])

print('Training model...\n')
clf = SVC(C = 1.0, kernel = 'linear').fit(tfidf_train, np.array(y_train))
print('Score:')
print(clf.score(tfidf_test, np.array(y_test)))

Training model...

Score:
0.659409020218


In [38]:
test = pd.read_csv('X_final_test.csv')

In [39]:
test.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,date,comment,commentNegative,commentPositive,uuid
0,20003635,406,4060501,62,"[{719: 'a9b7ba70783b617e9998dc4dd82eb3c5'}, {1...",e033ded0ac328c61d6b007139ec4e219,2012-04-16,"Согласна с Сашей и Наташей. Самая лучшая,самая...",NaN,NaN,e0880553-b7f1-4eaa-a800-966133bda683
1,20000782,411,4110104,13,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",9f269ce1c8e74e7b1e63f55b856068cd,2010-12-11,"Хороший пылесос. Вроде качественный. Да, шумно...",NaN,NaN,30039f3f-e719-4b47-b588-9a05fd11a799
2,20021828,413,4130401,63,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",dc9731cd5ea3ab7e16401cc678d046a5,2012-11-02,"Пользовалась им почти 2 года, без нареканий. Н...",NaN,NaN,352a5abc-10f4-4668-ae5a-a4285413811a
3,20027435,413,4130301,678,"[{539: '47c7a779688d5f67c55a42811da13210'}, {3...",c9b5ecdf0b853aab02103542187ea208,2016-04-22,До этого у меня был триммер всего с одной наса...,NaN,NaN,bfe4564d-944c-4680-b1ce-14e5486ada52
4,20004128,413,4130401,1466,"[{162: 'f932bed2d12442d21507b51d22b88dd7'}, {2...",32bf3da6fec001612d3e5048d87c5c64,2009-10-22,Удобны в использовании. Волосы становятся глад...,NaN,NaN,13547ee0-3400-422f-b831-528caeda9c69


In [30]:
clf = SVC(kernel = 'linear').fit(tfidf_train, np.array(y_train))
print('Score:')
print(clf.score(tfidf_test, np.array(y_test)))

Score:
0.669129082426


In [34]:
cont_features = df.columns[6:-1]
X_train, X_test, y_train, y_test = train_test_split(df.drop(['reting'], axis=1), 
df['reting'], test_size=0.33, random_state=241)

tfidf = TfidfVectorizer(analyzer = 'word', ngram_range=(1,3)).fit(X_train['parsed'])
tfidf_train = tfidf.transform(X_train['parsed'])
tfidf_test = tfidf.transform(X_test['parsed'])


cont_features_train = X_train[cont_features]
cont_features_test = X_test[cont_features]
from scipy.sparse import hstack
tfidf_data_train = hstack((tfidf_train, cont_features_train)).toarray()
tfidf_data_test = hstack((tfidf_test, cont_features_test)).toarray()

In [35]:
clf = SVC(kernel = 'linear').fit(tfidf_train, np.array(y_train))
print('Score:')
print(clf.score(tfidf_test, np.array(y_test)))

Score:
0.65299377916
